In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.self_supervised import *

Loading imports


In [2]:
torch.cuda.set_device(1)

In [3]:
dls = get_ss_data(192, splits=Meta.splits_stg1)

In [4]:
size=256
aug = get_aug_pipe(size, min_scale=0.4, mult=2, p_lighting=.9, p_affine=.9)
aug2 = get_aug_pipe(size, min_scale=0.3, mult=2, p_lighting=1.0, p_affine=1.0)
cb = SSCallback(BatchContrastiveLoss(5.0), size=size, aug_targ=aug, aug_pos=aug2, combined_loss=False)
learn = cnn_learner(dls, xresnet18, loss_func=get_loss(), lr=3e-3,
            opt_func=Adam, metrics=[], cbs=cb)

In [5]:
name = 'self_supervised_train_2_nocombined'

In [6]:
learn.load(f'runs/self_supervised_train_1-1')
do_fit(learn, 4, 1e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,time
0,0.015020,0.052774,53:04
1,0.012345,0.044440,53:20
2,0.008615,0.051786,53:09
3,0.007366,0.051274,53:43


In [6]:
learn.load(f'runs/self_supervised_train_1-1')
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 

In [ ]:
learn.load(f'runs/{name}-1')
learn.dls = get_ss_data(96, splits=Meta.splits_stg1, img_dir=path_jpg)
cb.update_size(384)
do_fit(learn, 4, 4e-3)
learn.save(f'runs/{name}-2')